In [ ]:
#STREAMS2TABLE
#written by Zachary J.J. Roper

#A script for converting raw event codes and timestamps into a trial-item dataset

In [24]:
#This notebook requires the following modules
import os
import numpy
import csv

In [ ]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [25]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)

In [ ]:
#The following provides indexed lists for each of the individual event codes of interest 

In [71]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '888':
        Saccade.append(i)

In [74]:
float(times[76])-float(times[70])

20.19730000000004

In [72]:
for i, x in enumerate(Saccade):
    print(Saccade[i],TargetOnset[i])

76 13
141 70
206 135
270 200
335 264
400 329
516 394
577 510
642 571
707 636
768 701
878 762
1047 872
1112 1041
1177 1106
1242 1171
1307 1236
1368 1301
1433 1362
1498 1427
1624 1492
1685 1557
1802 1618
1867 1679
1932 1796
1997 1861
2062 1926
2127 1991
2253 2056
2318 2121
2691 2186
2752 2247
2868 2312
2985 2685
3932 2746
3997 2862
4062 2979
4127 3147
4192 3926
4309 3991
4374 4056
5003 4121
5064 4186
5231 4303
5348 4368
5914 4997
5979 5058
6044 5225
6109 5342
6174 5908
6454 5973
6978 6038
7293 6103
7358 6168
7423 6335
7488 6448
7605 6972
7773 7287
7838 7352
7903 7417
7968 7482
8033 7599
8098 7767
8211 7832
8838 7897
8903 7962
8967 8027
9084 8092
9149 8205
9214 8832
9279 8897
9344 8961
9409 9078
9628 9143
9689 9208
9750 9273
9866 9338
9927 9403
9992 9622
10108 9683
10225 9744
10342 9860
10407 9921
10472 9986
10537 10102
10602 10219
10717 10336
10884 10401
11102 10466
11163 10531
11228 10596
11344 10711
11460 10878
11576 11096
11641 11157
11767 11222
11832 11338
11949 11454
12014 11570
120

In [29]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [81]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [52]:
print(TimeTrials[884])

[' 227590.6273', ' 227590.6396', ' 227590.6560', ' 227690.5779', ' 227690.5902', ' 227690.7786', ' 227690.7909', ' 227708.3791', ' 227708.3914', ' 227708.5757', ' 227708.5880', ' 227718.7789', ' 227718.7912', ' 227718.9837', ' 227718.9960', ' 227719.0083', ' 227778.9901', ' 227779.0024', ' 227779.0188', ' 227779.0310', ' 227779.0433', ' 227779.0638', ' 227779.1785', ' 227779.1908', ' 227779.2072', ' 227779.2195', ' 227779.2358', ' 227779.2481', ' 227779.3792', ' 227779.3915', ' 227779.4079', ' 227779.4202', ' 227779.4365', ' 227779.4488', ' 227779.5758', ' 227779.5922', ' 227779.6086', ' 227779.6209', ' 227779.6331', ' 227779.6454', ' 227779.7765', ' 227779.7929', ' 227779.8052', ' 227779.8216', ' 227779.8339', ' 227779.8461', ' 227779.9772', ' 227779.9936', ' 227780.0059', ' 227780.0223', ' 227780.0346', ' 227780.0468', ' 227780.1779', ' 227780.1943', ' 227780.2066', ' 227780.2230', ' 227780.2353', ' 227780.2475', ' 227780.3786', ' 227780.3909', ' 227780.4073', ' 227780.4196', ' 22778

In [32]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [48]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2757' in trial:
        catch[i] = 1
    else:
        catch[i] = 0

In [89]:
RT = [None]*len(Trials)

for i, trial in enumerate(Trials):
    print(trial)
    #if '888' in trial:
        #print(i+1,trial[0:5])
        #for g, spans in enumerate()
        #RT[i] = TimeTrials[i] - TimeTrials

['2301', '0', '2660', '0', '2651', '0', '2300', '0', '2001', '776', '1667', '2998', '3999', '0', '4001', '4052', '4060', '4108', '4152', '0', '4201', '4258', '4300', '4350', '4400', '0', '5135', '5680', '3800', '4650', '4663', '0', '4673', '4700', '4800', '4901', '6000', '0', '3003', '3200', '5000', '11000', '3100', '0', '3600', '3100', '3030', '4500', '4000', '0', '4500', '4000', '4690', '4790', '2999', '0']
['2301', '0', '2660', '0', '2651', '0', '2300', '0', '2810', '0', '888', '0', '2600', '0', '2001', '777', '0', '2727', '1667', '2998', '3999', '4001', '0', '4052', '4060', '4108', '4152', '4201', '0', '4258', '4300', '4350', '4400', '5180', '0', '5725', '3800', '4650', '4663', '4673', '0', '4700', '4800', '4901', '6000', '3007', '0', '3200', '5000', '11000', '3100', '3600', '0', '3100', '3030', '4500', '4000', '4500', '0', '4000', '4690', '4790', '2999', '0']
['2301', '0', '2660', '0', '2651', '0', '2300', '0', '2810', '0', '888', '0', '2600', '0', '2001', '777', '0', '2727', '166

In [49]:
checksum = sum(error)+sum(abort)+sum(catch)

In [50]:
checksum

325

In [45]:
for i,x in enumerate(abort):
    print(i+1,abort[i],accuracy[i],error[i])

1 0 0 0
2 0 1 0
3 0 1 0
4 0 1 0
5 0 1 0
6 0 1 0
7 0 1 0
8 1 0 0
9 0 0 0
10 0 1 0
11 0 1 0
12 0 0 0
13 0 0 0
14 1 0 0
15 0 1 0
16 1 0 0
17 1 0 0
18 0 1 0
19 0 1 0
20 0 1 0
21 0 1 0
22 0 0 0
23 0 1 0
24 0 1 0
25 0 1 0
26 0 0 1
27 0 0 0
28 0 1 0
29 1 0 0
30 0 1 0
31 0 1 0
32 0 1 0
33 0 1 0
34 0 1 0
35 0 1 0
36 0 0 1
37 0 1 0
38 0 1 0
39 1 0 0
40 1 0 0
41 1 0 0
42 1 0 0
43 1 0 0
44 1 0 0
45 0 0 0
46 0 1 0
47 1 0 0
48 0 1 0
49 1 0 0
50 0 1 0
51 1 0 0
52 1 0 0
53 0 0 1
54 1 0 0
55 1 0 0
56 1 0 0
57 1 0 0
58 1 0 0
59 1 0 0
60 1 0 0
61 1 0 0
62 1 0 0
63 1 0 0
64 1 0 0
65 1 0 0
66 1 0 0
67 1 0 0
68 0 1 0
69 0 1 0
70 0 1 0
71 0 1 0
72 0 1 0
73 1 0 0
74 0 1 0
75 0 1 0
76 1 0 0
77 1 0 0
78 1 0 0
79 1 0 0
80 1 0 0
81 1 0 0
82 1 0 0
83 1 0 0
84 1 0 0
85 1 0 0
86 1 0 0
87 0 0 0
88 0 1 0
89 1 0 0
90 1 0 0
91 0 1 0
92 1 0 0
93 0 0 0
94 1 0 0
95 1 0 0
96 1 0 0
97 1 0 0
98 1 0 0
99 1 0 0
100 1 0 0
101 1 0 0
102 1 0 0
103 1 0 0
104 0 1 0
105 0 1 0
106 0 1 0
107 0 1 0
108 0 1 0
109 1 0 0
110 1 0 0
111 0 0 